
<a href="https://colab.research.google.com/github/OsipovOleg/crash-python-notebooks/blob/master/scientific_computing.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Байесовская регрессия

Пусть дана обучающая выборка $\mathcal{D}$, состоящая из $N$ элементов, которая получена следующим образом: 

$$
y_i = w_0 + x_i w_1 + N(0, \sigma^2).
$$

Пусть априорное распределение для каждого из весов является нормальным. 

1. Получите вид апостериорного распределения $p(w| \mathcal{D})$.
2. Рассмотрите как меняется апостериорное распреление весов в зависимости *от числа объектов*, которые мы используем для обучения модели. 
- Получите параметры этого распределения. 
- Постройте плотности распределения весов. 
- Просемлируйте несколько значений весов из полученного апостериорного распределения и постройте предсказания (линии регересии на некотором интервале).
3. Как ведут себя полученные линии при добавлении новых данных. Какие выводы можно сделать. 